In [ ]:
#!pip install geemap[all]
#!pip install earthengine-api
#!pip install ee

import geemap,geemap as import_map
import ee
import os
ee.Authenticate()
ee.Initialize(project='geobvieer')
vis_map=import_map.Map()

In [ ]:
e_g_district_b=ee.FeatureCollection('projects/geobvieer/assets/east_godavari')
vis_map.addLayer(e_g_district_b,{},'East Godavari District Boundary')
modis_data=ee.ImageCollection("MODIS/061/MOD13A1").select("NDVI")

In [ ]:
region = ee.Geometry.Polygon([[
   [80.87606575641922, 16.306551451427744],
    [82.60457886502209, 16.306551451427744],
    [82.60457886502209, 18.01443579097642],
    [80.87606575641922, 18.01443579097642],
    [80.87606575641922, 16.306551451427744]

]])

In [ ]:
landsat_india = modis_data.map(lambda img:img.set('doy', ee.Date(img.get('system:time_start')).getRelative('day', 'year')))
doy=landsat_india.filterDate('2014-01-31','2024-01-31')
filter=ee.Filter.equals(leftField='doy',rightField='doy')
join=ee.Join.saveAll('doy_matches')
joincol=ee.ImageCollection(join.apply(doy,landsat_india,filter))
stack=joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()))


In [ ]:

ndviVis = {
  'min': 0,
  'max': 30000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
}

In [ ]:
rgbvis=stack.map(lambda img:img.visualize(bands=['NDVI_mean'],**ndviVis).clip(e_g_district_b))
gifparams={
    'region':region,
    'dimensions':338,
    'crs':'EPSG:4326',
    'framesPerSecond':25,
    'format':'gif'
}
print(rgbvis.getVideoThumbURL(gifparams))